# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('city_df.csv')


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[['Lat', 'Lng']].values.tolist()
weights = city_df['Humidity']
# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
    'zoom_level': 12,
    'center': (0, 0)
}
fig = gmaps.figure(layout=figure_layout)
# Add the layer to the map
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Convert Temperature from C to F
city_df_voc = city_df.copy()
city_df_voc = city_df_voc[
    (city_df_voc['Max Temp'] < 80)\
    & (city_df_voc['Max Temp'] > 70)\
    & (city_df_voc['Max Temp'].notnull())\
    & (city_df_voc['Wind Speed'] < 10)\
    & (city_df_voc['Wind Speed'].notnull())\
    & (city_df_voc['Cloudiness'] == 0 )\
    & (city_df_voc['Cloudiness'].notnull())\
].copy()
city_df_voc
    

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
80,olinda,-8.0089,-34.8553,78.800,69,0,3.09,BR,1619308984
144,dalbandin,28.8947,64.4101,74.804,10,0,2.15,PK,1619309329
164,howard springs,-12.4970,131.0470,78.800,34,0,3.09,AU,1619309440
208,karratha,-20.7377,116.8463,79.808,28,0,5.67,AU,1619309543
380,oussouye,12.4850,-16.5469,73.400,88,0,2.06,SN,1619309974
398,hervey bay,-25.2986,152.8535,75.002,66,0,1.79,AU,1619310018
422,buraydah,26.3260,43.9750,73.400,28,0,3.09,SA,1619310074
443,touros,-5.1989,-35.4608,77.000,88,0,2.06,BR,1619310128
465,koraput,18.8167,82.7167,72.986,48,0,1.61,IN,1619310193
486,esna,25.2934,32.5540,77.000,20,0,4.01,EG,1619310251


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# geocoordinates
target_radius = 5000
target_type = "hotel"
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
res = []
for idx, row in city_df_voc.iterrows():
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key,
    }
    time.sleep(2)
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        response_json = response.json()
        if len(response_json['results']) > 0:
            print(f"{row['City']} is successfully responded")
            res.append({'Hotel Name': response_json['results'][0]['name'],
                       'City': row['City'],
                        'Country': row['Country'],
                        'Lat': row['Lat'],
                        'Lng': row['Lng'],
                       })
hotel_df = pd.DataFrame(res)
hotel_df

olinda is successfully responded
dalbandin is successfully responded
howard springs is successfully responded
karratha is successfully responded
oussouye is successfully responded
hervey bay is successfully responded
buraydah is successfully responded
touros is successfully responded
koraput is successfully responded
esna is successfully responded
imbituba is successfully responded
tahta is successfully responded


,Hotel Name,City,Country,Lat,Lng
0,Olinda,olinda,BR,-8.0089,-34.8553
1,Dalbandin,dalbandin,PK,28.8947,64.4101
2,Darwin,howard springs,AU,-12.4970,131.0470
3,Karratha,karratha,AU,-20.7377,116.8463
4,Oussouye,oussouye,SN,12.4850,-16.5469
5,Hervey Bay,hervey bay,AU,-25.2986,152.8535
6,Buraydah,buraydah,SA,26.3260,43.9750
7,Touros,touros,BR,-5.1989,-35.4608
8,Koraput,koraput,IN,18.8167,82.7167
9,Esna,esna,EG,25.2934,32.5540


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig


# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…